<a href="https://colab.research.google.com/github/fuluki/awesome-notebooks-SEO/blob/master/PPC_and_GSC_Data_Analysis_Script_%5BPublic%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""PPC and GSC Data Analysis

# How to use:
1. Export Google Ads data:
   * Export your PPC conversion data report
   * Save as 'google_ads_data.csv'
2. Export Search Console data:
   * Export last 3 months of GSC data
   * Save as 'search_console_data.csv'
3. Upload both files to Colab using the files panel on the left
4. Click Run (the play button)
5. Download your results

# Output:
* Script generates 'analyzed_results.xlsx' with three sheets:
   * All Data: Complete merged dataset
   * Positions 1-3: Keywords ranking in positions 1-3
   * Positions 4-10: Keywords ranking in positions 4-10
"""

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from scipy import stats

def clean_keyword(keyword):
    return re.sub(r'[\[\]\"]', '', str(keyword))

def process_ppc_data(ppc_file):
    df = pd.read_csv(ppc_file, skiprows=2)
    df = df.rename(columns={'Search keyword report': 'Keyword'})
    df['Keyword'] = df['Keyword'].apply(clean_keyword)
    return df

def process_gsc_data(gsc_file):
    df = pd.read_csv(gsc_file)
    return df[['Top queries', 'Position']].rename(columns={'Top queries': 'Keyword'})

def calculate_position_revenue_potential(df):
    """Calculate potential revenue increase based on position improvements"""
    def estimate_ctr_improvement(current_position):
        if pd.isna(current_position):
            return 0
        position_ctr = {1: 1.0, 2: 0.8, 3: 0.6, 4: 0.5, 5: 0.4}
        current_ctr = position_ctr.get(int(round(current_position)), 0.3)
        potential_ctr = position_ctr.get(1)
        return (potential_ctr - current_ctr) / current_ctr if current_ctr > 0 else 0

    df['Potential_CTR_Improvement'] = df['Position'].apply(estimate_ctr_improvement)
    # Convert Conv. value to numeric, replacing any non-numeric values with 0
    df['Conv. value'] = pd.to_numeric(df['Conv. value'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
    df['Revenue_Potential'] = df['Conv. value'] * df['Potential_CTR_Improvement']
    return df

def generate_statistical_insights(df):
    """Generate statistical analysis of key metrics"""
    stats_df = df.agg({
        'Conv. value': ['mean', 'median', 'std', 'count'],
        'Position': ['mean', 'median', 'std'],
        'Cost': ['sum', 'mean', 'median'],
        'Revenue_Potential': ['sum', 'mean', 'median']
    }).round(2)

    # Add correlation analysis
    correlations = df[['Position', 'Conv. value', 'Cost']].corr().round(3)
    return stats_df, correlations

def create_visualizations(df, output_prefix):
    """Generate visualization plots"""
    # Position vs Conversion Value
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df, x='Position', y='Conv. value', alpha=0.5)
    plt.title('Position vs Conversion Value')
    plt.tight_layout()
    plt.savefig(f'{output_prefix}_position_value.png')
    plt.close()

    # Revenue Potential by Position
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, x=df['Position'].round(), y='Revenue_Potential')
    plt.title('Revenue Potential by Position')
    plt.tight_layout()
    plt.savefig(f'{output_prefix}_revenue_potential.png')
    plt.close()

def analyze_positions(ppc_df, gsc_df):
    merged_df = pd.merge(ppc_df, gsc_df, on='Keyword', how='left')
    merged_df = calculate_position_revenue_potential(merged_df)

    pos_1_3 = merged_df[(merged_df['Position'] >= 1) & (merged_df['Position'] <= 3)]
    pos_4_10 = merged_df[(merged_df['Position'] > 3) & (merged_df['Position'] <= 10)]

    # Generate insights for each segment
    all_stats, all_corr = generate_statistical_insights(merged_df)
    p1_3_stats, p1_3_corr = generate_statistical_insights(pos_1_3)
    p4_10_stats, p4_10_corr = generate_statistical_insights(pos_4_10)

    return {
        'all_data': merged_df,
        'positions_1_3': pos_1_3,
        'positions_4_10': pos_4_10,
        'stats': {
            'all': (all_stats, all_corr),
            'pos_1_3': (p1_3_stats, p1_3_corr),
            'pos_4_10': (p4_10_stats, p4_10_corr)
        }
    }

def save_results(results, output_file):
    with pd.ExcelWriter(output_file) as writer:
        # Data sheets
        results['all_data'].to_excel(writer, sheet_name='All Data', index=False)
        results['positions_1_3'].to_excel(writer, sheet_name='Positions 1-3', index=False)
        results['positions_4_10'].to_excel(writer, sheet_name='Positions 4-10', index=False)

        # Statistics sheets
        for segment, (stats, corr) in results['stats'].items():
            stats.to_excel(writer, sheet_name=f'{segment}_stats')
            corr.to_excel(writer, sheet_name=f'{segment}_correlations')

def main():
    ppc_file = 'google_ads_data.csv'
    gsc_file = 'search_console_data.csv'
    output_file = 'analyzed_results.xlsx'

    ppc_df = process_ppc_data(ppc_file)
    gsc_df = process_gsc_data(gsc_file)

    results = analyze_positions(ppc_df, gsc_df)
    save_results(results, output_file)

    # Generate visualizations
    create_visualizations(results['all_data'], 'all')
    create_visualizations(results['positions_1_3'], 'pos_1_3')
    create_visualizations(results['positions_4_10'], 'pos_4_10')

if __name__ == '__main__':
    main()